In [6]:
from UTIL.network import find_free_port_no_repeat

port, release_fn = find_free_port_no_repeat()

new port: 60459


In [ ]:
def arrange_id(N_AGENT_EACH_TEAM):
    N_AGENT_EACH_TEAM = np.array(N_AGENT_EACH_TEAM) 
    AGENT_ID_EACH_TEAM_cv = []
    begin = 0
    for _, n in enumerate(N_AGENT_EACH_TEAM):
        b = begin
        s = begin + n
        AGENT_ID_EACH_TEAM_cv.append(range(b, s))
        begin = s
    return AGENT_ID_EACH_TEAM_cv
arrange_id([25,25])


--2022-09-16 16:40:35--  http://cloud.fuqingxu.top:4080/remote.php/dav/files/fuqingxu/keys/notebook_old.pub
Resolving cloud.fuqingxu.top (cloud.fuqingxu.top)... 43.154.70.224
Connecting to cloud.fuqingxu.top (cloud.fuqingxu.top)|43.154.70.224|:4080... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Nextcloud", charset="UTF-8"
Reusing existing connection to cloud.fuqingxu.top:4080.
HTTP request sent, awaiting response... 200 OK
Length: 403 [application/octet-stream]
Saving to: ‘./TEMP/_xkey’

./TEMP/_xkey        100%[===================>]     403  --.-KB/s    in 0s      

2022-09-16 16:40:38 (76.4 MB/s) - ‘./TEMP/_xkey’ saved [403/403]

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCnPRZXNNBttb6XvGf/W5rf9YKKCXCjzffhAV241Gbx/m/KA3niQR/1Y5U9YC3S+Paia2wvVJpKX1dTOwJKCIZuWvBv4ynHgaQ4occrrc2t4SAW1BzD/YeQM+Y/KrqRQ38emUartiYVDzgsCq1euE4pw1dESh9uazv9pZyS3ieEz+UVCiDeyeXXcI3hlKba7ARLA15txrxp/1em4T8nnGsPaWsuF/pXGmoYO+6P4nlWKXgJxvaHrEZUZSIR0JNOUjLPNj

In [168]:
import numpy as np
ckpg_info = [
    {'win_rate':0.0},
    {'win_rate':0.2},
    {'win_rate':0.2},
    {'win_rate':0.5},
    {'win_rate':0.7},
    {'win_rate':0.8},
    {'win_rate':0.9},
]

def random_select():
    """randomly select a group index

    Args:
        AlgorithmConfig.hete_same_prob: a probability about choosing the frontier net as the teammate

    Returns:
        int: a group index
    """
    # when random win rate is high, direct to frontend nets
    hete_same_prob = 0.0
    if np.random.rand() < hete_same_prob:
        return 0
    
    n_option = len(ckpg_info)
    rand_winrate = np.random.randint(low=1, high=n_option+1)
        
    return rand_winrate

print(random_select())

7


In [53]:
import torch, copy
import numpy as np
"""
    improve np.reshape and torch.view function
    If a dim is assigned with 0, it will keep its original dimension
    eg.1    x.shape = (4, 5, 6, 7); new_shape = [0, 0, -1]
            y = my_view(x, new_shape)
            y.shape = (4, 5, 6*7)

    eg.2    x.shape = (4, 5, 6, 7); new_shape = [-1, 0, 0]
            y = my_view(x, new_shape)
            y.shape = (4*5, 6, 7)

    eg.3    x.shape = (4, 5, 6); new_shape = [0, 0, -1, 3]
            y = my_view(x, new_shape)
            y.shape = [4, 5, 2, 3]

    eg.4    x.shape = (3, 4, 5, 6); new_shape = [0, 2, -1, 0, 0]
            y = my_view(x, new_shape)
            y.shape = [3, 2, 2, 5, 6]


"""
def my_view(x, shape):
    if -1 in shape[1:-1]: return my_view_test(x, shape)
    reverse_lookup = True if shape[0] == -1 else False
    if not reverse_lookup:
        for i, dim in enumerate(shape):
            if dim == 0:
                shape[i] = x.shape[i]
    else:
        for i in range(len(shape)):
            ni = -(i + 1)  # iter -1,-2,-3,...
            dim = shape[ni]
            if dim == 0:
                shape[ni] = x.shape[ni]
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)

# def my_view_test(x, shape):
#     # fill both way until meet -1 
#     for i, dim in enumerate(shape):
#         if dim == 0: shape[i] = x.shape[i]
#         elif dim == -1: break
#         elif dim != x.shape[i]: break
#     for i in range(len(shape)):
#         ni = -(i + 1); dim = shape[ni]
#         if dim == 0: shape[ni] = x.shape[ni]
#         elif dim == -1: break
#     # print(shape)
#     if isinstance(x, np.ndarray):
#         return x.reshape(*shape)
#     return x.view(*shape)

def my_view_test(x, shape):
    # fill both way until meet -1 
    for i, dim in enumerate(shape):
        if dim == 0: shape[i] = x.shape[i]
        elif dim == -1: break
        elif dim != x.shape[i]: break
    for i in range(len(shape)):
        ni = -(i + 1); dim = shape[ni]
        if dim == 0: shape[ni] = x.shape[ni]
        elif dim == -1: break
    # print(shape)
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)


x = torch.rand(4, 5, 6, 7); 
new_shape = [0, 0, -1]
print((my_view_test(x, copy.copy(new_shape))==my_view(x, copy.copy(new_shape))).all())

x = torch.rand(4, 5, 6, 7); 
new_shape = [-1, 0, 0]
print((my_view_test(x, copy.copy(new_shape))==my_view(x, copy.copy(new_shape))).all())


x = torch.rand(4, 5, 6); 
new_shape = [0, 0, -1, 3]
print((my_view_test(x, copy.copy(new_shape))==my_view(x, copy.copy(new_shape))).all())


x = torch.rand(3, 4, 5, 6); 
new_shape = [0, 2, -1, 0, 0]
print((my_view_test(x, copy.copy(new_shape))==my_view(x, copy.copy(new_shape))).all())


x = torch.rand(4, 5, 6, 7)
new_shape = [0,30,0]
print(my_view_test(x, copy.copy(new_shape)).shape)

x = torch.rand(4, 6, 5, 7)
new_shape = [0, 3, -1, 0, 0]
print(my_view_test(x, copy.copy(new_shape)).shape)

x = torch.rand(4, 6, 5, 7)
new_shape = [0, 3, 2, 0, 0]
print(my_view_test(x, copy.copy(new_shape)).shape)

x = torch.rand(4, 6, 5, 9)
new_shape = [0, -1, 45]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,6,45]')

x = torch.rand(4, 6, 5, 9, 2)
new_shape = [0, -1, 90]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,6,90]')
# x = torch.rand(4, 6, 5, 7)
# new_shape = [0, 3, 0, 0, 0] # error
# print(my_view_test(x, copy.copy(new_shape)).shape)

# 规则： 异变数字(!=, -1)最多两个，如果是两个，则必须相连

# x = torch.rand(4, 5, 6, 7, 8, 9)
# new_shape = [0,30,0,-1] # error
# print(my_view_test(x, copy.copy(new_shape)).shape)


x = torch.rand(4, 6, 5, 9, 2, 5)
new_shape = [0, -1, 90, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,6,90, 5]')

x = torch.rand(4, 6, 5, 9, 2, 5)
new_shape = [0, 0, 90, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,6,90, 5]')

x = torch.rand(4, 6, 5, 9, 2, 5)
new_shape = [0, 30, 18, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18, 5]')

x = torch.rand(4, 6, 5, 9, 2)
new_shape = [0, 30, 18]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18]')

x = torch.rand(6, 5, 9, 2, 5)
new_shape = [30, 18, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[30,18, 5]')


x = torch.rand(4, 6, 5, 9, 2, 5)
new_shape = [0, -1, 18, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18, 5]')

x = torch.rand(4, 6, 5, 9, 2)
new_shape = [0, -1, 18]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18]')

x = torch.rand(6, 5, 9, 2, 5)
new_shape = [-1, 18, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[30,18, 5]')

x = torch.rand(4, 6, 5, 9, 2, 5)
new_shape = [0, 30, -1, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18, 5]')

x = torch.rand(4, 6, 5, 9, 2)
new_shape = [0, 30, -1]
print((my_view_test(x, copy.copy(new_shape)).shape), '[4,30,18]')

x = torch.rand(6, 5, 9, 2, 5)
new_shape = [30, -1, 0]
print((my_view_test(x, copy.copy(new_shape)).shape), '[30,18, 5]')
'''
            x.shape = (3, 4, 5, 6); new_shape = [12, 0, -1]
            Error: 12(!=3) and -1 must stick together!
            Fix: new_shape = [12, 0, 0]
            Fix: new_shape = [12, -1, 6]
            Fix: new_shape = [12, -1, 0]
'''
x = torch.rand(3, 4, 5, 6)
new_shape = [12, 0, 0]
new_shape = [12, -1, 6]
new_shape = [12, -1, 0]
print((my_view_test(x, copy.copy(new_shape)).shape))


tensor(True)
tensor(True)
tensor(True)
tensor(True)
torch.Size([4, 30, 7])
torch.Size([4, 3, 2, 5, 7])
torch.Size([4, 3, 2, 5, 7])
torch.Size([4, 6, 45]) [4,6,45]
torch.Size([4, 6, 90]) [4,6,90]
torch.Size([4, 6, 90, 5]) [4,6,90, 5]
torch.Size([4, 6, 90, 5]) [4,6,90, 5]
torch.Size([4, 30, 18, 5]) [4,30,18, 5]
torch.Size([4, 30, 18]) [4,30,18]
torch.Size([30, 18, 5]) [30,18, 5]
torch.Size([4, 30, 18, 5]) [4,30,18, 5]
torch.Size([4, 30, 18]) [4,30,18]
torch.Size([30, 18, 5]) [30,18, 5]
torch.Size([4, 30, 18, 5]) [4,30,18, 5]
torch.Size([4, 30, 18]) [4,30,18]
torch.Size([30, 18, 5]) [30,18, 5]
torch.Size([12, 5, 6])


In [20]:
my_view_test(x, new_shape)==my_view(x, new_shape)

tensor([[[True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, 

In [6]:
import torch, time
import numpy as np
def tear_apart(x, n_digit):
    out = np.zeros(n_digit, dtype=x.dtype)
    p = n_digit
    for _ in range(n_digit):
        p-=1
        tmp = x % 10
        out[p] = tmp
        x = x // 10
    return out

tear_apart(np.array([10086]), 10)

array([0, 0, 0, 0, 0, 1, 0, 0, 8, 6])

In [109]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [181]:

%%cython
import numpy as np
cimport numpy as np
cimport cython
np.import_array()


ctypedef fused DTYPE_all_t:
    np.float32_t
    np.float64_t
    np.int64_t
    np.int32_t  # to compat Wi
ctypedef fused DTYPE_t:
    np.float32_t
    np.float64_t


ctypedef fused DTYPE_intlong_t:
    np.int64_t
    np.int32_t  # to compat Windows
    
    
# x: input
# n_digit: output dimension
# base: 进制
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def tear_number_apart(np.float64_t x, DTYPE_intlong_t n_digit, DTYPE_intlong_t base=16, DTYPE_all_t mv_left=8):
    cdef np.ndarray out = np.zeros(n_digit, dtype=float)
    cdef int p = n_digit
    cdef float tmp = 0
    reverse = x < 0

    cdef float m_init = base
    if reverse: x = -x
    m_init = m_init ** mv_left
    x = x * m_init
    for _ in range(n_digit):
        p -= 1
        if p==0: 
            out[p] = x
            break
        tmp = x % base
        out[p] = tmp
        x = x // base
    if reverse: out = -out
    return out



# [-1.  9.  9.  9.  9.  9.  8.  8.  0.  0.]
# -1200.0
# parts = tear_number_apart(-1200, n_digit=10, base=10, mv_left=0)
# print(parts)
# comb_num_back(parts, n_digit=10, base=10, mv_left=0)


# def put_number_together(parts, n_digit, base):
#     out = 0
#     z = n_digit//2
#     mp = base**(n_digit - n_digit//2 - 1)
#     for p in range(n_digit):
#         out += parts[p] * mp
#         mp = mp / base
        
#     return out
def comb_num_back(arr, n_digit, base, mv_left):
    out = 0
    tmp = base ** (n_digit - mv_left - 1)
    for x in arr:
        out += x * tmp
        tmp = tmp/base
        
    return out

def tear_num_arr(arr, DTYPE_intlong_t n_digit, DTYPE_intlong_t base, DTYPE_all_t mv_left):
    return np.concatenate([tear_number_apart(x, n_digit, base, mv_left) for x in arr], axis=0)

parts = tear_number_apart(-1200, n_digit=10, base=10, mv_left=0)
print(parts)
print(comb_num_back(parts, n_digit=10, base=10, mv_left=0))


In file included from /home/hmp/.local/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /home/hmp/.local/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/hmp/.local/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/hmp/.cache/ipython/cython/_cython_magic_05262793914bae67f251919f56f690a1.c:683:
/home/hmp/.local/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


[-0. -0. -0. -0. -0. -0. -1. -2. -0. -0.]
-1200.0


In [272]:
for i in range(1000):
    q = (np.random.rand() - 0.5)*1e3
    parts = tear_number_apart(q, n_digit=10, base=10, mv_left=0)
    print(q, parts)
    res = np.abs(comb_num_back(parts, n_digit=10, base=10, mv_left=0)-q) < 1e-6
    if not res:
        print('??? np.abs(comb_num_back(parts, n_digit=10, base=10, mv_left=0)-q)', np.abs(comb_num_back(parts, n_digit=10, base=10, mv_left=0)-q))
        assert False

230.82640393813236 [0.         0.         0.         0.         0.         0.
 0.         2.         3.         0.82640392]
492.6766723569501 [0.         0.         0.         0.         0.         0.
 0.         4.         9.         2.67667246]
59.956808644741066 [0.         0.         0.         0.         0.         0.
 0.         0.         5.         9.95680904]
-59.651731736030555 [-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -5.         -9.65173149]
-238.3352484366702 [-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -2.         -3.         -8.33524799]
5.7337278365590105 [0.         0.         0.         0.         0.         0.
 0.         0.         0.         5.73372793]
275.885923960165 [0.         0.         0.         0.         0.         0.
 0.         2.         7.         5.88592386]
-98.07044541018584 [-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -9.  

In [184]:
parts = tear_number_apart(-1200, n_digit=10, base=10, mv_left=0)
print(parts)
print(comb_num_back(parts, n_digit=10, base=10, mv_left=0))


[-0. -0. -0. -0. -0. -0. -1. -2. -0. -0.]
-1200.0


In [185]:


parts = tear_number_apart(255, n_digit=10, base=2, mv_left=1)
print(parts)
comb_num_back(parts, n_digit=10, base=2, mv_left=1)

[0. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


255.0

client.send_targeted_dgram('ddd')

In [186]:


parts = tear_number_apart(255.778, n_digit=10, base=10, mv_left=-1)
print(parts)
comb_num_back(parts, n_digit=10, base=10, mv_left=-1)

[0.         0.         0.         0.         0.         0.
 0.         0.         2.         5.57780027]


255.77800273895264

In [187]:

parts = tear_number_apart(-1200, n_digit=10, base=10, mv_left=0)
print(parts)
comb_num_back(parts, n_digit=10, base=10, mv_left=0)

[-0. -0. -0. -0. -0. -0. -1. -2. -0. -0.]


-1200.0

In [54]:
import torch
from torch.distributions.categorical import Categorical
from torch.nn.functional import kl_div
import torch.nn.functional as F
from UTIL.tensor_ops import repeat_at
probs = torch.Tensor(
    [
        [0.4, 0.6],
        [0.3, 0.7],
        [0.3, 0.7]
    ]
    )


# probs.S # (?, n_agent, n_action)
n_agent = probs.shape[-2]
probs_rep = repeat_at(tensor=probs, insert_dim=-2, n_times=n_agent)
# probs_rep.S # (?, n_agent, n_agent, n_action)
probs_rep_transpose = probs_rep.swapaxes(-2,-3)
mat = (probs_rep*probs_rep.log()-probs_rep*probs_rep_transpose.log()).sum(-1)
mat # (?, n_agent, n_agent)
# F.kl_div(probs_rep.log(), probs_rep_transpose, reduction='batchmean')



tensor([[0.0000, 0.0226, 0.0226],
        [0.0216, 0.0000, 0.0000],
        [0.0216, 0.0000, 0.0000]])

In [110]:
# n_agent = 16
tree = get_division_tree(18)
# current_level = 3
# blood_distance = np.ones(shape=(n_agent,n_agent), dtype=np.float64) * np.nan
# for i in range(n_agent):
#     for j in range(n_agent):
#         if i==j:blood_distance[i,j] = 0
#         if not np.isnan(blood_distance[i,j]):
#             continue
#         for t in range(current_level+1):
#             investigate_level = (current_level) - t
#             if tree[investigate_level, i] == tree[investigate_level, j]:
#                 blood_distance[i,j] = t
#                 blood_distance[j,i] = t
#                 break


blood_distance = get_blood_distance(tree, 2)
print(blood_distance)
print(tree)

[[0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0.

In [72]:

[[ 0.  2.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [ 2.  0.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [ 2.  2.  0. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  0. nan  2. nan  2.  2. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  0. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  0. nan  2.  2. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  0. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  0.  2. nan  2. nan  2.  2. nan  2.]
 [nan nan nan  2. nan  2. nan  2.  0. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  0. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  0. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  2. nan  0. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  0.  2. nan  2.]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  2.  0. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  0. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  2.  2. nan  0.]]

 

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  1,  1,  1,  1,  0,  1,  0,  0,  0,  1,  0,  0],
       [ 1,  0,  3,  2,  3,  1,  1,  1,  2,  3,  0,  2,  2,  3,  0,  0],
       [ 1,  0,  3,  5,  7,  6,  1,  6,  2,  7,  4,  5,  2,  3,  4,  0],
       [12,  0,  3, 11, 15,  6,  1, 13, 10,  7,  9,  5,  2, 14,  4,  8]])

In [2]:
import pickle
with open('TEMP/wifi', 'rb') as f:
    p_list = pickle.load(f)

p_list

{'d1': tensor([1.1618e-13, 4.3868e-16, 3.4678e-18, 0.0000e+00, 1.0000e+00, 8.3701e-14,
         1.8273e-12], device='cuda:0', requires_grad=True),
 'd2': tensor([0.0756, 0.2109, 0.0628, 0.0104, 0.3506, 0.1273, 0.1624],
        device='cuda:0', requires_grad=True)}

In [32]:
import torch
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.nn.functional import kl_div


logits1 = torch.Tensor([  4.4786,  -1.1005,  -5.9407, -15.7646,  5.2623,   4.1508,   7.2341]).type(torch.DoubleTensor)
logits2 = torch.Tensor([ -0.5480,  0.4779, -0.7330, -2.5290,  0.9862, -0.0273,  0.2163]).type(torch.DoubleTensor)

dist1 = Categorical(logits = logits1)
dist2 = Categorical(logits = logits2)
print(dist1.probs)
print(dist2.probs)
from torch.distributions import kl_divergence

print(kl_divergence(dist1, dist2))
print(kl_divergence(dist2, dist1))


tensor([5.0909e-02, 1.9223e-04, 1.5197e-06, 8.2279e-11, 1.1147e-01, 3.6681e-02,
        8.0075e-01], dtype=torch.float64)
tensor([0.0756, 0.2109, 0.0628, 0.0104, 0.3506, 0.1273, 0.1624],
       dtype=torch.float64)
tensor(1.0829, dtype=torch.float64)
tensor(2.6698, dtype=torch.float64)


In [7]:
# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

tensor([1.1618e-13, 4.3868e-16, 3.4678e-18, 0.0000e+00, 1.0000e+00, 8.3701e-14,
        1.8273e-12], device='cuda:0', requires_grad=True)

In [64]:
import torch
import numpy as np
from UTIL.tensor_ops import gather_righthand, repeat_at
def scatter_righthand(scatter_into, src, index, check=True):
    index = index.long()
    i_dim = index.dim()
    s_dim = src.dim()
    t_dim = i_dim - 1
    index_new_shape = list(src.shape)
    index_new_shape[t_dim] = index.shape[t_dim]
    for _ in range(i_dim, s_dim):
        index = index.unsqueeze(-1)
    index_expand = index.expand(index_new_shape)  # only this two line matters
    return scatter_into.scatter(t_dim, index_expand, src)

orig = torch.Tensor([[[ 0,  1,  2], [ 3,  4,  5]],
                    [[ 6,  7,  8], [ 9, 10, 11]],
                    [[12, 13, 14], [15, 16, 17]]])
index = torch.Tensor([[0], [1], [0]])
print(orig)

res      = gather_righthand(src=orig, index=index)
res[:] = 0

orig_fix = orig.clone().detach()
orig_fix = scatter_righthand(orig_fix, src=res, index=index)
print(orig_fix)


# orig.scatter(dim=1,index=index.long(),src=res)

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]],

        [[12., 13., 14.],
         [15., 16., 17.]]])
tensor([[[ 0.,  0.,  0.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [15., 16., 17.]]])


In [71]:
index = torch.randint(high=16,size=(64,5))
print('s1')
res = gather_righthand(src,index)
print('s1')
res[:] = 0
print('s2')
resX = scatter_righthand(scatter_into=src, src=res, index=index)
print('s2')
print(resX.S)


s1
s1
s2
s2
torch.Size([64, 16, 8, 88, 888])


In [3]:
import torch
import numpy as np
from UTIL.tensor_ops import gather_righthand, repeat_at
from UTIL.tensor_ops import add_onehot_id_at_last_dim, repeat_at, _2tensor, gather_righthand, scatter_righthand
x_in = torch.Tensor([
    [1,1,1],
    [2,2,2],
    [3,3,3],
    [4,4,4],
])
n_agent = 4


nets = [
    lambda x: x+100,
    lambda x: x+200,
    lambda x: x+300,
    lambda x: x+400
]


x0 = add_onehot_id_at_last_dim(x_in)
# x1 = self.shared_net(x0)
res = []
for i in range(n_agent):
    res.append(nets[i](x0[..., i, :]))
x2 = torch.stack(res, -2)
# x22 = self.nets[0](x1)
print('x2',x2)

######### forward twice: shuffle forward
perm_index = torch.randperm(n_agent, device=x_in.device)   # shape = (n_agent)
print('perm_index',perm_index)

perm_index = perm_index.expand(x_in.shape[:-1]) # shape = (...?, n_agent)
# x_in shape = (...?, n_agent, coredim)
perm_x_in = gather_righthand(src=x_in, index=perm_index, check=False)
perm_x0 = add_onehot_id_at_last_dim(perm_x_in)
perm_res = []
for i in range(n_agent):
    perm_res.append(nets[i](perm_x0[..., i, :]))
perm_x2 = torch.stack(perm_res, -2)

# 103
# 202
# 304
# 401


# 401 vs 101
# 202 vs 202
# 103 vs 303
# 304 vs 404

confact_x2 = torch.zeros_like(perm_x2) + np.nan
confact_x2 = scatter_righthand(scatter_into=confact_x2, src=perm_x2, index=perm_index)
print('confact_x2',confact_x2)

x2 tensor([[101., 101., 101., 101., 100., 100., 100.],
        [202., 202., 202., 200., 201., 200., 200.],
        [303., 303., 303., 300., 300., 301., 300.],
        [404., 404., 404., 400., 400., 400., 401.]])
perm_index tensor([2, 1, 3, 0])
confact_x2 tensor([[401., 401., 401., 400., 400., 400., 401.],
        [202., 202., 202., 200., 201., 200., 200.],
        [103., 103., 103., 101., 100., 100., 100.],
        [304., 304., 304., 300., 300., 301., 300.]])


In [4]:
import torch

res=torch.isnan(torch.tensor([1, float('inf'), 2, float('-inf'), float('nan')]))
print(res)

res=torch.isfinite(torch.tensor([1, float('inf'), 2, float('-inf'), float('nan')]))
print(res)







tensor([False, False, False, False,  True])
tensor([ True, False,  True, False, False])
